In [ ]:
# %run "/content/drive/MyDrive/Colab Notebooks/spark-setup.ipynb"

In [ ]:
import pyspark.sql.functions as F
import re
import os
year_pattern = re.compile(r'(19[0-9]{2}|20[0-2][0-9])')

In [ ]:
csv_dir = '/content/drive/MyDrive/datasets/olympics/'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
olympics_df = None

for csv_file in csv_files:
    # Load the CSV file into a DataFrame
    year = None
    match = year_pattern.search(csv_file)
    if match:
        year = match.group(0)  # Extracted year from the file name

    file_path = os.path.join(csv_dir, csv_file)
    temp_df = spark.read.csv(file_path, header=True, inferSchema=True)
    temp_df = temp_df.withColumn('Year', F.lit(year))

    # Combine the data into one DataFrame (using union)
    if olympics_df is None:
        olympics_df = temp_df
    else:
        olympics_df = olympics_df.union(temp_df)

In [ ]:
olympics_df.show()

In [ ]:
spark.sql("use olympics_db")

In [ ]:
olympics_df.write.mode("overwrite").format("delta").saveAsTable("olympics")